### Income and Outlays Monthly Data

In [2]:
import sys
import json
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### After-tax income growth

In [6]:
data = fred_df('A229RX0', start='1988')['VALUE']
data2 = (data.pct_change(12) * 100).dropna()
data2.to_csv(data_dir / 'rdpigrowth.csv', header=['rdpi'], index_label='date')
node = end_node(data2, 'blue!80!cyan', date=True)
write_txt(text_dir / 'rdpi_node.txt', node)

ltdate = dtxt(data2.index[-1])['mon1']
ltval = data2.iloc[-1]
prdate = dtxt(data2.index[-2])['mon1']
prval = data2.iloc[-2]
prdate2 = dtxt(data2.index[-13])['mon1']
prval2 = data2.iloc[-13]
ltavg = data2.iloc[-13:].mean()
ltavg2 = data2.loc['2017-03-01': '2020-02-01'].mean()

text = ('The Bureau of Economic Analysis '+
        '\href{https://www.bea.gov/data/income-saving/personal-income}{report} '+
        'an inflation-adjusted one-year change in after-tax income per person (see '+
        '{\color{blue!80!cyan}\\textbf{---}}) of '+
        f'{ltval:.1f} percent in {ltdate}, {prval:.1f} percent '+
        f'in {prdate}, and {prval2:.1f} percent in {prdate2}. '+
        f'Over the past year, the measure has averaged {ltavg:.1f} percent. '+
        'During the three years before the COVID-19 pandemic, per capita '+
        f'after-tax income grew at an average annual rate of {ltavg2:.1f} percent.')
write_txt(text_dir / 'rdpigrowth.txt', text)
print(text)

The Bureau of Economic Analysis \href{https://www.bea.gov/data/income-saving/personal-income}{report} an inflation-adjusted one-year change in after-tax income per person (see {\color{blue!80!cyan}\textbf{---}}) of 7.9 percent in July 2020, 8.1 percent in June 2020, and 1.1 percent in July 2019. Over the past year, the measure has averaged 4.0 percent. During the three-years prior to the COVID-19 pandemic, per capita after-tax income grew at an average annual rate of 2.4 percent.


### Monthly consumer spending growth

In [7]:
data = pd.read_csv('../data/rpcepop_hist.csv', index_col='DATE', parse_dates=True)

series = {'PCEC96': 'PCE', 
          'POPTHM': 'POP'}

df = pd.DataFrame()
for srs, name in series.items():
    url = f'http://research.stlouisfed.org/fred2/series/{srs}/downloaddata/{srs}.csv'
    s = pd.read_csv(url, index_col='DATE', parse_dates=True)['VALUE']
    df[name] = s
    
df['PCEPOP'] = df['PCE'] / df['POP']
data2 = data['rpcepop'].append((df.PCEPOP.pct_change(12) * 100).dropna().rename('rpcepop'))

data2.to_csv(data_dir / 'pcegrowth.csv', header=['rpcepop'], index_label='date')

node = end_node(data2, 'green!80!black', date=True)
write_txt(text_dir / 'pcegrowth_node.txt', node)

In [8]:
s = pd.read_csv(data_dir / 'pcegrowth.csv', parse_dates=['date']).set_index('date')['rpcepop']
d = series_info(s)
val_lt = data2.iloc[-1]
inc_dec = inc_dec_percent(val_lt)
val_yr = data2.iloc[-13]
inc_dec2 = inc_dec_percent(val_yr, how='of')
compare = compare_text(val_lt, val_yr, [0.1, 0.5, 2.0])

text = ('Consumer spending is also \href{https://www.bea.gov/data/consumer-spending/main}{reported}'+
        f' on a monthly basis. Inflation- and population-adjusted consumer spending {inc_dec}'+
        f'over the year ending {d["date_latest_ft"]} '+
        '(see {\color{green!80!black}\\textbf{---}}), '+
        f'{compare} the previous year rate ({inc_dec2} for the year ending '+
        f'{d["date_year_ago_ft"]}). ')

write_txt(text_dir / 'pcegrowth.txt', text)

text

'Consumer spending is also \\href{https://www.bea.gov/data/consumer-spending/main}{reported} on a monthly basis. Inflation- and population-adjusted consumer spending decreased by 4.2 percentover the year ending July 2020 (see {\\color{green!80!black}\\textbf{---}}), far below the previous year rate (an increase of 2.0 percent for the year ending July 2019). '

### Personal saving rate

In [2]:
data = fred_df('PSAVERT')['VALUE']

data.to_csv(data_dir / 'psavert.csv', index_label='date', header=True)

datelt = dtxt(data.index[-1])['mon1']
latest = data.iloc[-1]
ch_covid = latest - data.loc['2020-02-01']

txt = val_inc_pp(ch_covid)

text = (f'As of {datelt}, the Bureau of Economic Analysis '+
        '\href{https://www.bea.gov/data/income-saving/personal-saving-rate}{report} a rate '+
        f'of personal saving of {latest:.1f} percent '+
        '(see {\color{red}\\textbf{---}}). Since February 2020, '+
        f'the personal saving rate {txt}.')

write_txt(text_dir / 'psavert.txt', text)

print(text)

node = end_node(data, 'red', date=True)
write_txt(text_dir / 'psavert_node.txt', node)

As of July 2020, the Bureau of Economic Analysis \href{https://www.bea.gov/data/income-saving/personal-saving-rate}{report} a rate of personal saving of 17.8 percent (see {\color{red}\textbf{---}}). Since February 2020, the personal saving rate increased by a total of 9.5 percentage points.


### Personal Income

In [6]:
series = {
 'TOTAL': '\hspace{2mm}Personal income',
 'LABOR': '\hspace{-1mm}\cbox{green!75!black} Labor',
 'A034RC': '\hspace{4mm} Wages and salaries',
 'A038RC': '\hspace{4mm} Supplements',
 'CAPITAL': '\hspace{-1mm}\cbox{orange!40!yellow}Capital',
 'A041RC': "\hspace{4mm} Proprietors' income",
 'A048RC': '\hspace{4mm} Rental income',
 'A064RC': '\hspace{4mm} Personal interest income',
 'B703RC': '\hspace{4mm} Personal dividend income',
 'TRANSFER': '\hspace{-1mm}\cbox{blue!80!white}Welfare',
 'W823RC': '\hspace{4mm} Social security',
 'W824RC': '\hspace{4mm} Medicare',
 'W729RC': '\hspace{4mm} Medicaid',
 'W825RC': '\hspace{4mm} Unemployment insurance',
 'W826RC': "\hspace{4mm} Veterans' benefits",
 'W827RC': '\hspace{4mm} Other',
 'A061RC': '\hspace{4mm} Less welfare contributions'}

# Components of request
base = f'https://apps.bea.gov/api/data/?&UserID={bea_key}'
dset = '&method=GetData&datasetname=NIPA'
tbl = 'T20600' 
freq = '&Frequency=M'
yr = ','.join(map(str, range(1988, 2021)))
fmt = '&ResultFormat=json'
url = f'{base}{dset}&TableName={tbl}{freq}&Year={yr}{fmt}'

# Request data
r = requests.get(url).json()['BEAAPI']['Results']

df = pd.DataFrame(r['Data'])
df['Date'] = pd.to_datetime(df['TimePeriod'], format='%YM%m')

df = (df.set_index(['Date', 'SeriesCode'])['DataValue'].str.replace(',', '')
        .astype(float).unstack()
        .assign(CAPITAL = lambda x: x['A041RC'] + x['A048RC'] + x['W210RC'],
                TRANSFER = lambda x: x['A577RC'] - x['A061RC'],
                A061RC = lambda x: - x['A061RC'])
        .rename({'A065RC': 'TOTAL', 'A033RC': 'LABOR'}, axis=1))

df.to_csv(data_dir / 'pi_raw.csv', index_label='date')

In [7]:
df = pd.read_csv(data_dir / 'pi_raw.csv').set_index('date')

pce = fred_df('PCEPI', start='1988')['VALUE']
pce = (pce.iloc[-1] / pce)

pop = df['B230RC']

data = (df.div(pop, axis=0).multiply(pce, axis=0))

(data.loc['1989':,['LABOR', 'CAPITAL', 'TRANSFER']]
     .to_csv(data_dir / 'pi_levels.csv', index_label='date'))

tbl = ((data[series.keys()].rename(series, axis=1)
            .iloc[[-1, -2, -3, -4, -13, -37]].T * 1000)
            .applymap('{:,.0f}'.format))

tbl.columns = [dtxt(c)['mon6'] for c in tbl.columns]
tbl.index.name = ''

tbl.to_csv(data_dir / 'pi_levels.tex', sep='&', line_terminator='\\\ ', quotechar=' ')

ld = data.index[-1]
ltdate = dtxt(data.index[-1])['mon1']
totval = data.loc[ld, 'TOTAL'] * 1000
labval = data.loc[ld, 'LABOR'] * 1000
capval = data.loc[ld, 'CAPITAL'] * 1000
welval = data.loc[ld, 'TRANSFER'] * 1000

text = (f'In {ltdate}, annualized personal income is '+
        f'\${totval:,.0f} per capita. Labor income totals \${labval:,.0f} '+
        f'per person; capital and proprietor income is \${capval:,.0f} '+
        f'per person; and welfare or transfer income is \${welval:,.0f} '+
        'per person. ')

write_txt(text_dir / 'pi_levels_basic.txt', text)
write_txt(text_dir / 'pi_date.txt', ltdate)
print(text)

In July 2020, annualized personal income is \$60,699 per capita. Labor income totals \$34,159 per person; capital and proprietor income is \$15,970 per person; and welfare or transfer income is \$10,569 per person. 
